Links to Yelp dataset files : https://drive.google.com/drive/folders/15_pQBF8zIOnxXSaDx19DN8eU-lXrvIeW?usp=share_link https://drive.google.com/drive/folders/1-9SOeLW8g97fiX2On7_7Wm1ePt2pSnDU?usp=share_link

# Preprocessing...

In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import matplotlib.pyplot as plt
import os
import tqdm
import pickle
from pathlib import Path
from torch.utils.data import DataLoader
from sklearn.metrics import ndcg_score
np.random.seed(0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing Amazon ...


In [ ]:
user_thresh=20
feature_thresh=2000
review_dir='/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/Yelp/Kaggle/yelp_academic_dataset_review.json'
sentires_dir='/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/Yelp/Yelp'
test_length=5
sample_ratio=2
val_length=1
neg_length=100
dataset='yelp'
save_path='/content/drive/MyDrive/Yelp/'
save_path2='/content/drive/MyDrive/Yelp/'

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

np.random.seed(0)

In [ ]:
def get_feature_list(sentiment_data):
    """
    from user sentiment data, get all the features [F1, F2, ..., Fk] mentioned in the reviews
    :param sentiment_data: [user, item, [feature1, opinion1, sentiment1], [feature2, opinion2, sentiment2] ...]
    :return: feature set F
    """
    feature_list = []
    for row in sentiment_data:
        for fos in row[2:]:
            feature = fos[0]
            if feature not in feature_list:
                feature_list.append(feature)
    feature_list = np.array(feature_list)
    return feature_list

In [ ]:
def get_user_attention_matrix(sentiment_data, user_num, feature_list, max_range=5):
    """
    build user attention matrix
    :param sentiment_data: [user, item, [feature1, opinion1, sentiment1], [feature2, opinion2, sentiment2] ...]
    :param user_num: number of users
    :param feature_list: [F1, F2, ..., Fk]
    :param max_range: normalize the attention value to [1, max_range]
    :return: the user attention matrix, Xij is user i's attention on feature j
    """
    user_counting_matrix = np.zeros((user_num, len(feature_list)))  # tij = x if user i mention feature j x times
    for row in sentiment_data:
        user = row[0]
        for fos in row[2:]:
            feature = fos[0]
            user_counting_matrix[user, feature] += 1
    user_attention_matrix = np.zeros((user_num, len(feature_list)))  # xij = [1-N], normalized attention matrix
    for i in range(len(user_counting_matrix)):
        for j in range(len(user_counting_matrix[i])):
            if user_counting_matrix[i, j] == 0:
                norm_v = 0  # if nor mentioned: 0
            else:
                norm_v = 1 + (max_range - 1) * ((2 / (1 + np.exp(-user_counting_matrix[i, j]))) - 1)  # norm score
            user_attention_matrix[i, j] = norm_v
    user_attention_matrix = np.array(user_attention_matrix, dtype='float32')
    return user_attention_matrix

In [ ]:
def get_item_quality_matrix(sentiment_data, item_num, feature_list, max_range=5):
    """
    build item quality matrix
    :param sentiment_data: [user, item, [feature1, opinion1, sentiment1], [feature2, opinion2, sentiment2] ...]
    :param item_num: number of items
    :param feature_list: [F1, F2, ..., Fk]
    :param max_range: normalize the quality value to [1, max_range]
    :return: the item quality matrix, Yij is item i's quality on feature j
    """
    item_counting_matrix = np.zeros((item_num, len(feature_list)))  # kij = x if item i's feature j is mentioned x times
    item_sentiment_matrix = np.zeros((item_num, len(feature_list)))  # sij = x if the overall rating is x (sum up)
    for row in sentiment_data:
        item = row[1]
        for fos in row[2:]:
            feature = fos[0]
            sentiment = fos[2]
            item_counting_matrix[item, feature] += 1
            if sentiment == '+1':
                item_sentiment_matrix[item, feature] += 1
            elif sentiment == '-1':
                item_sentiment_matrix[item, feature] -= 1
            else:
                print("sentiment data error: the sentiment value can only be +1 or -1")
                exit(1)
    item_quality_matrix = np.zeros((item_num, len(feature_list)))
    for i in range(len(item_counting_matrix)):
        for j in range(len(item_counting_matrix[i])):
            if item_counting_matrix[i, j] == 0:
                norm_v = 0  # if not mentioned: 0
            else:
                norm_v = 1 + ((max_range - 1) / (1 + np.exp(-item_sentiment_matrix[i, j])))  # norm score
            item_quality_matrix[i, j] = norm_v
    item_quality_matrix = np.array(item_quality_matrix, dtype='float32')
    return item_quality_matrix

In [ ]:
def get_user_item_dict(sentiment_data):
    """
    build user & item dictionary
    :param sentiment_data: [user, item, [feature1, opinion1, sentiment1], [feature2, opinion2, sentiment2] ...]
    :return: user dictionary {u1:[i, i, i...], u2:[i, i, i...]}, similarly, item dictionary
    """
    user_dict = {}
    item_dict = {}
    for row in sentiment_data:
        user = row[0]
        item = row[1]
        if user not in user_dict:
            user_dict[user] = [item]
        else:
            user_dict[user].append(item)
        if item not in item_dict:
            item_dict[item] = [user]
        else:
            item_dict[item].append(user)
    return user_dict, item_dict

In [ ]:
def get_user_item_set(sentiment_data):
    """
    get user item set
    :param sentiment_data: [user, item, [feature1, opinion1, sentiment1], [feature2, opinion2, sentiment2] ...]
    :return: user_set = set(u1, u2, ..., um); item_set = (i1, i2, ..., in)
    """
    user_set = set()
    item_set = set()
    for row in sentiment_data:
        user = row[0]
        item = row[1]
        user_set.add(user)
        item_set.add(item)
    return user_set, item_set

In [ ]:
def sample_training_pairs(user, training_items, item_set, sample_ratio=10):
    positive_items = set(training_items)
    negative_items = set()
    for item in item_set:
        if item not in positive_items:
            negative_items.add(item)
    neg_length = len(positive_items) * sample_ratio
    negative_items = np.random.choice(np.array(list(negative_items)), neg_length, replace=False)
    train_pairs = []
    for p_item in positive_items:
        train_pairs.append([user, p_item, 1])
    for n_item in negative_items:
        train_pairs.append([user, n_item, 0])
    return train_pairs

In [ ]:
def check_string(string):
    # if the string contains letters
    string_lowercase = string.lower()
    contains_letters = string_lowercase.islower()
    return contains_letters

In [ ]:
def visualization(train_losses, val_losses, path):
    plt.plot(np.arange(len(train_losses)), train_losses, label='training loss')
    plt.plot(np.array(len(val_losses)), val_losses, label='validation loss')
    plt.legend()
    plt.savefig(path)
    plt.clf()

In [ ]:
def get_mask_vec(user_attantion, k):
    """
    get the top-k mask for features. The counterfactual explanations can only be chosen from this space
    :param user_attantion: user's attantion vector on all the features
    :param k: the k from mask
    :return: a mask vector with 1's on the top-k features that the user cares about and 0's for others.
    """
    top_indices = np.argsort(user_attantion)[::-1][:k]
    mask = [0 for i in range(len(user_attantion))]
    for index in top_indices:
        if user_attantion[index] > 0:  # only consider the user mentioned features
            mask[index] = 1
    return np.array(mask)

In [ ]:
def feature_filtering(sentiment_data, valid_features):
    """
    filter the sentiment data, remove the invalid features
    :param sentiment_data: [userID, itemID, [fos triplet 1], [fos triplet 2], ...]
    :param valid_features: set of valid features
    :return: the filtered sentiment data
    """
    cleaned_sentiment_data = []
    for row in sentiment_data:
        user = row[0]
        item = row[1]
        cleaned_sentiment_data.append([user, item])
        for fos in row[2:]:
            if fos[0] in valid_features:
                cleaned_sentiment_data[-1].append(fos)
        if len(cleaned_sentiment_data[-1]) == 2:
            del cleaned_sentiment_data[-1]
    return np.array(cleaned_sentiment_data)

In [ ]:
def sentiment_data_filtering(sentiment_data, user_thresh, feature_thresh):
    """
    filter the sentiment data, remove the users with less review number less than "user_thresh" and remove the features
    mentioned less than "feature_thresh" or don't contain letters.
    :param sentiment_data: [userID, itemID, [fos triplet 1], [fos triplet 2], ...]
    :param user_thresh: the threshold for user reviews
    :param feature_thresh: the threshold features
    :return: the filtered sentiment data
    """
    print('======================= filtering sentiment data =======================')
    sentiment_data = np.array(sentiment_data)
    last_length = len(sentiment_data)
    un_change_count = 0  # iteratively filtering users and features, if the data stay unchanged twice, stop
    user_dict, item_dict = get_user_item_dict(sentiment_data)
    features = get_feature_list(sentiment_data)
    print("original review length: ", len(sentiment_data))
    print("original user length: ", len(user_dict))
    print("original item length: ", len(item_dict))
    print("original feature length: ", len(features))
    while True:
        # feature filtering
        feature_count_dict = {}
        for row in sentiment_data:
            for fos in row[2:]:
                feature = fos[0]
                if feature not in feature_count_dict:
                    feature_count_dict[feature] = 1
                else:
                    feature_count_dict[feature] += 1
        valid_features = set()
        for key, value in feature_count_dict.items():
            if check_string(key) and value > feature_thresh:
                valid_features.add(key)
        sentiment_data = [row for row in sentiment_data if row[2][0] in valid_features]
        sentiment_data = feature_filtering(sentiment_data, valid_features)
        length = len(sentiment_data)
        if length != last_length:
            last_length = length
            un_change_count = 0
        else:
            un_change_count += 1
            if un_change_count == 2:
                break
        # user filtering
        user_dict, item_dict = get_user_item_dict(sentiment_data)
        valid_user = set()  # the valid users
        for key, value in user_dict.items():
            if len(value) > (user_thresh - 1):
              valid_user.add(key)
        sentiment_data = [x for x in sentiment_data if x[0] in valid_user]  # remove user with small interactions
        length = len(sentiment_data)
        if length != last_length:
            last_length = length
            un_change_count = 0
        else:
            un_change_count += 1
            if un_change_count == 2:
                break
    user_dict, item_dict = get_user_item_dict(sentiment_data)
    features = get_feature_list(sentiment_data)
    print('valid review length: ', len(sentiment_data))
    print("valid user: ", len(user_dict))
    print('valid item : ', len(item_dict))
    print("valid feature length: ", len(features))
    print('user dense is:', len(sentiment_data) / len(user_dict))
    sentiment_data = np.array(sentiment_data)
    return sentiment_data

In [ ]:
class YelpDataset():
    def __init__(self):
        super().__init__()
        # self.args = preprocessing_args
        self.sentiment_data = None  # [userID, itemID, [fos triplet 1], [fos triplet 2], ...]

        self.user_name_dict = {}  # rename users to integer names
        self.item_name_dict = {}
        self.feature_name_dict = {}

        self.features = []  # feature list
        self.users = []
        self.items = []

        # the interacted items for each user, sorted with date {user:[i1, i2, i3, ...], user:[i1, i2, i3, ...]}
        self.user_hist_inter_dict = {}
        # the interacted users for each item
        self.item_hist_inter_dict = {}  

        self.user_num = None
        self.item_num = None
        self.feature_num = None  # number of features

        self.user_feature_matrix = None  # user aspect attention matrix
        self.item_feature_matrix = None  # item aspect quality matrix

        self.training_data = None
        self.test_data = None
        self.pre_processing()
        self.get_user_item_feature_matrix()
        self.sample_training()  # sample training data, for traning BPR loss
        self.sample_test()  # sample test data
    def pre_processing(self,):
       sentiment_data = []  # [userID, itemID, [fos triplet 1], [fos triplet 2], ...]
       with open(sentires_dir, 'r') as f:
            line = f.readline().strip()
            while line:
                # print(count)
                # print('line', line)
                user = line.split('@')[0]
                item = line.split('@')[1]
                sentiment_data.append([user, item])
                l = len(user) + len(item)
                fosr_data = line[l+3:]
                for seg in fosr_data.split('||'):
                    fos = seg.split(':')[0].strip('|')
                    if len(fos.split('|')) > 1:
                        feature = fos.split('|')[0]
                        opinion = fos.split('|')[1]
                        sentiment = fos.split('|')[2]
                        sentiment_data[-1].append([feature, opinion, sentiment])
                line = f.readline().strip()
       sentiment_data = np.array(sentiment_data)
       sentiment_data = sentiment_data_filtering(
          sentiment_data, 
          user_thresh, 
          feature_thresh)
       user_dict, item_dict = get_user_item_dict(sentiment_data)  # not sorted with time
       user_item_date_dict = {}   # {(user, item): date, (user, item): date ...}  # used to remove duplicate
       for i, line in enumerate(open(review_dir, "r")):
            record = json.loads(line)
            user = record['user_id']
            item = record['business_id']
            date = record['date']
            if user in user_dict and item in user_dict[user] and (user, item) not in user_item_date_dict:
                user_item_date_dict[(user, item)] = date
       sentiment_data = [row for row in sentiment_data if (row[0], row[1]) in user_item_date_dict]
       sentiment_data = sentiment_data_filtering(sentiment_data, user_thresh, feature_thresh)
       user_dict, item_dict = get_user_item_dict(sentiment_data)
       for key in list(user_item_date_dict.keys()):
            if key[0] not in user_dict or key[1] not in user_dict[key[0]]:
                del user_item_date_dict[key]
        
        # rename users, items, and features to integer names
       user_name_dict = {}
       item_name_dict = {}
       feature_name_dict = {}
       features = get_feature_list(sentiment_data)
        
       count = 0
       for user in user_dict:
            if user not in user_name_dict:
                user_name_dict[user] = count
                count += 1
       count = 0
       for item in item_dict:
            if item not in item_name_dict:
                item_name_dict[item] = count
                count += 1
       count = 0
       for feature in features:
            if feature not in feature_name_dict:
                feature_name_dict[feature] = count
                count += 1
        
       for i in range(len(sentiment_data)):
            sentiment_data[i][0] = user_name_dict[sentiment_data[i][0]]
            sentiment_data[i][1] = item_name_dict[sentiment_data[i][1]]
            for j in range(len(sentiment_data[i]) - 2):
                sentiment_data[i][j+2][0] = feature_name_dict[sentiment_data[i][j + 2][0]]

       renamed_user_item_date_dict = {}
       for key, value in user_item_date_dict.items():
            renamed_user_item_date_dict[user_name_dict[key[0]], item_name_dict[key[1]]] = value
       user_item_date_dict = renamed_user_item_date_dict

        # sort with date
       user_item_date_dict = dict(sorted(user_item_date_dict.items(), key=lambda item: item[1]))

       user_hist_inter_dict = {}  # {"u1": [i1, i2, i3, ...], "u2": [i1, i2, i3, ...]}, sort with time
       item_hist_inter_dict = {}
        # ranked_user_item_dict = {}  # {"u1": [i1, i2, i3, ...], "u2": [i1, i2, i3, ...]}
       for key, value in user_item_date_dict.items():
            user = key[0]
            item = key[1]
            if user not in user_hist_inter_dict:
                user_hist_inter_dict[user] = [item]
            else:
                user_hist_inter_dict[user].append(item)
            if item not in item_hist_inter_dict:
                item_hist_inter_dict[item] = [user]
            else:
                item_hist_inter_dict[item].append(user)

       user_hist_inter_dict = dict(sorted(user_hist_inter_dict.items()))
       item_hist_inter_dict = dict(sorted(item_hist_inter_dict.items()))

       users = list(user_hist_inter_dict.keys())
       items = list(item_hist_inter_dict.keys())
       self.sentiment_data = sentiment_data
       self.user_name_dict = user_name_dict
       self.item_name_dict = item_name_dict
       self.feature_name_dict = feature_name_dict
       self.user_hist_inter_dict = user_hist_inter_dict
       self.item_hist_inter_dict = item_hist_inter_dict
       self.users = users
       self.items = items
       self.features = features
       self.user_num = len(users)
       self.item_num = len(items)
       self.feature_num = len(features)
       return True
    def get_user_item_feature_matrix(self,):
        # exclude test data from the sentiment data to construct matrix
        train_u_i_set = set()
        for user, items in self.user_hist_inter_dict.items():
            items = items
            for item in items:
                train_u_i_set.add((user, item))
        train_sentiment_data = []
        for row in self.sentiment_data:
            user = row[0]
            item = row[1]
            if (user, item) in train_u_i_set:
                train_sentiment_data.append(row)
        self.user_feature_matrix = get_user_attention_matrix(
            train_sentiment_data, 
            self.user_num, 
            self.features, 
            max_range=5)
        self.item_feature_matrix = get_item_quality_matrix(
            train_sentiment_data, 
            self.item_num, 
            self.features, 
            max_range=5)
        return True
    def sample_training(self):
        print('======================= sample training data =======================')
        # print(self.user_feature_matrix.shape, self.item_feature_matrix.shape)
        training_data = []
        training_pairs = np.loadtxt(save_path2+'training_data_2.txt',dtype=str)
        for pair in training_pairs:
          if pair[0] in self.user_name_dict.keys() and pair[1] in self.item_name_dict.keys():
            training_data.append([self.user_name_dict[pair[0]],self.item_name_dict[pair[1]],int(pair[2])])

        # item_set = set(self.items)
        # for user, items in self.user_hist_inter_dict.items():
        #     items = items[:-(test_length+val_length)]
        #     training_pairs = sample_training_pairs(
        #         user, 
        #         items, 
        #         item_set, 
        #         sample_ratio)
        #     for pair in training_pairs:
        #         training_data.append(pair)
        print('# training samples :', len(training_data))
        self.training_data = np.array(training_data)
        return True
    
    def sample_test(self):
        print('======================= sample test data =======================')
        user_item_label_list = []  # [[u, [item1, item2, ...], [l1, l2, ...]], ...]
        with open(save_path2+'test_data_2.pickle', 'rb') as f:
            test_pairs= pickle.load(f)
        for user_id in test_pairs.keys():
          if user_id in self.user_name_dict.keys():
            user=self.user_name_dict[user_id]
            items_ids=test_pairs[user_id][0]
            labels=test_pairs[user_id][1]
            items=np.array([self.item_name_dict[item] for item in items_ids if item in self.item_name_dict.keys()])
            labels=np.array([float(labels[i]) for i in range(len(labels)) if items_ids[i] in self.item_name_dict.keys()])
            user_item_label_list.append([user,items,labels])
        
        # for user, items in self.user_hist_inter_dict.items():
        #     items = items[-(test_length+val_length):]
        #     user_item_label_list.append([user, items, np.ones(len(items))])  # add the test items
        #     negative_items = [item for item in self.items if 
        #         item not in self.user_hist_inter_dict[user]]  # the not interacted items
        #     negative_items = np.random.choice(np.array(negative_items), neg_length, replace=False)
        #     user_item_label_list[-1][1] = np.concatenate((user_item_label_list[-1][1], negative_items), axis=0)
        #     user_item_label_list[-1][2] = np.concatenate((user_item_label_list[-1][2], np.zeros(neg_length)), axis=0)
        print('# test samples :', len(user_item_label_list))
        self.test_data = np.array(user_item_label_list)
        user_item_label_list2 = []  # [[u, [item1, item2, ...], [l1, l2, ...]], ...]
        with open(save_path2+'validation_data_2.pickle', 'rb') as f:
            validation_pairs= pickle.load(f)
        for user_id in validation_pairs.keys():
          if user_id in self.user_name_dict.keys():
            user=self.user_name_dict[user_id]
            items_ids=validation_pairs[user_id][0]
            labels=validation_pairs[user_id][1]
            items=np.array([self.item_name_dict[item] for item in items_ids if item in self.item_name_dict.keys()])
            labels=np.array([float(labels[i]) for i in range(len(labels)) if items_ids[i] in self.item_name_dict.keys()])
            user_item_label_list2.append([user,items,labels])
        
        # for user, items in self.user_hist_inter_dict.items():
        #     items = items[-(test_length+val_length):]
        #     user_item_label_list.append([user, items, np.ones(len(items))])  # add the test items
        #     negative_items = [item for item in self.items if 
        #         item not in self.user_hist_inter_dict[user]]  # the not interacted items
        #     negative_items = np.random.choice(np.array(negative_items), neg_length, replace=False)
        #     user_item_label_list[-1][1] = np.concatenate((user_item_label_list[-1][1], negative_items), axis=0)
        #     user_item_label_list[-1][2] = np.concatenate((user_item_label_list[-1][2], np.zeros(neg_length)), axis=0)
        print('# validation samples :', len(user_item_label_list2))
        self.validation_data = np.array(user_item_label_list2)
        return True
    
    def save(self, save_path):
        return True
    
    def load(self):
        return False

In [ ]:
def yelp_preprocessing():
    rec_dataset = YelpDataset()
    return rec_dataset

In [ ]:
def dataset_init():
	if dataset == "yelp":
		rec_dataset = yelp_preprocessing()
	elif dataset == "cell_phones" or "kindle_store" or "electronic" or "cds_and_vinyl":
		rec_dataset = amazon_preprocessing()
	return rec_dataset

# Training base Model...

In [ ]:
dataset="yelp"
gpu=True
cuda='0'
weight_decay=0.000001
lr=0.005
epochs=36
batch_size=128
rec_k=5

In [ ]:
import numpy as np
from torch.utils.data import Dataset
class UserItemInterDataset(Dataset):
    def __init__(self, data, user_feature_matrix, item_feature_matrix):
        self.data = data
        self.user_feature_matrix = user_feature_matrix
        self.item_feature_matrix = item_feature_matrix

    def __getitem__(self, index):
        user = self.data[index][0]
        item = self.data[index][1]
        label = self.data[index][2]
        user_feature = self.user_feature_matrix[user]
        item_feature = self.item_feature_matrix[item]
        return user_feature, item_feature, label
    def __len__(self):
        return len(self.data)

# Train Black-box model...

In [ ]:
import numpy as np
import torch
from sklearn.metrics import ndcg_score

In [ ]:
def compute_ndcg(test_data, user_feature_matrix, item_feature_matrix, k, model, device):
    model.eval()
    ndcgs = []
    with torch.no_grad():
        for row in test_data:
            user = row[0]
            items = row[1]
            gt_labels = row[2]
            # print(len(items))
            # print(len(gt_labels))
            user_features = np.array([user_feature_matrix[user] for i in range(len(items))])
            item_features = np.array([item_feature_matrix[item] for item in items])
            scores = model(torch.from_numpy(user_features).to(device),
                                    torch.from_numpy(item_features).to(device)).squeeze()
        
            scores = np.array(scores.to('cpu'))
            # print(user_features.shape)
            # print(len(scores))
            # print(len(gt_labels))
            ndcg = ndcg_score([gt_labels], [scores], k=k)
            ndcgs.append(ndcg)
    ave_ndcg = np.mean(ndcgs)
    return ave_ndcg


In [ ]:
import torch
import numpy as np
import os
import tqdm
import pickle
from pathlib import Path
from torch.utils.data import DataLoader

In [ ]:
from numpy import core

class BaseRecModel(torch.nn.Module):
    def __init__(self, feature_length):
        super(BaseRecModel, self).__init__()
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(feature_length * 2, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, user_feature, item_feature):
        fusion = torch.cat((user_feature, item_feature), 1)
        out = self.fc(fusion)
        return out


In [ ]:
if gpu:
  device = torch.device('cuda:%s' % cuda)
else:
  device = 'cpu'
print(device)

cuda:0


In [ ]:
rec_dataset = dataset_init()

<ipython-input-57-04712ae582c0>:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentiment_data = np.array(sentiment_data)


======================= filtering sentiment data =======================
original review length:  1366842
original user length:  84842
original item length:  52594
original feature length:  11135


<ipython-input-14-8047dbd5a64f>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(cleaned_sentiment_data)


valid review length:  696465
valid user:  13034
valid item :  47031
valid feature length:  255
user dense is: 53.43447905477981


<ipython-input-15-394a3508aa51>:66: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentiment_data = np.array(sentiment_data)


======================= filtering sentiment data =======================


<ipython-input-15-394a3508aa51>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentiment_data = np.array(sentiment_data)


original review length:  696465
original user length:  13034
original item length:  47031
original feature length:  255
valid review length:  696465
valid user:  13034
valid item :  47031
valid feature length:  255
user dense is: 53.43447905477981
======================= sample training data =======================
# training samples : 317566
======================= sample test data =======================
# test samples : 2049
# validation samples : 2049


<ipython-input-57-04712ae582c0>:213: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.test_data = np.array(user_item_label_list)
<ipython-input-57-04712ae582c0>:235: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.validation_data = np.array(user_item_label_list2)


In [ ]:
Path(save_path).mkdir(parents=True, exist_ok=True)
with open(os.path.join(save_path2,dataset + "_dataset_obj_main.pickle"), 'wb') as outp:
    pickle.dump(rec_dataset, outp, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(os.path.join(save_path2, dataset + "_dataset_obj_main.pickle"), 'rb') as inp:
  rec_dataset = pickle.load(inp)

In [ ]:
print(len(list(set(rec_dataset.features))))

255


In [ ]:
rec_dataset.test_data

In [ ]:
def train_base_recommendation():
    if gpu:
        device = torch.device('cuda:%s' % cuda)
    else:
        device = 'cpu'

    # rec_dataset = dataset_init()
    Path(save_path).mkdir(parents=True, exist_ok=True)
    with open(os.path.join(save_path2,dataset + "_dataset_obj_main.pickle"), 'wb') as outp:
        pickle.dump(rec_dataset, outp, pickle.HIGHEST_PROTOCOL)

    train_loader = DataLoader(dataset=UserItemInterDataset(rec_dataset.training_data, 
                                rec_dataset.user_feature_matrix, 
                                rec_dataset.item_feature_matrix),
                          batch_size=batch_size,
                          shuffle=True)

    model = BaseRecModel(rec_dataset.feature_num).to(device)
    loss_fn = torch.nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)

    out_path = os.path.join("./logs", dataset + "_logs")
    Path(out_path).mkdir(parents=True, exist_ok=True)

    ndcg = compute_ndcg(rec_dataset.test_data, 
            rec_dataset.user_feature_matrix, 
            rec_dataset.item_feature_matrix, 
            rec_k, 
            model, 
            device)
    print('init ndcg:', ndcg)
    for epoch in tqdm.trange(epochs):
        model.train()
        optimizer.zero_grad()
        losses = []
        for user_behaviour_feature, item_aspect_feature, label in train_loader:
            user_behaviour_feature = user_behaviour_feature.to(device)
            item_aspect_feature = item_aspect_feature.to(device)
            label = label.float().to(device)
            out = model(user_behaviour_feature, item_aspect_feature).squeeze()
            loss = loss_fn(out, label)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            losses.append(loss.to('cpu').detach().numpy())
            ave_train = np.mean(np.array(losses))
            # print(out)
            # print(label)
            # print('----------')
        print('epoch %d: ' % epoch, 'training loss: ', ave_train)
        # compute necg
        if epoch % 1 == 0:
            ndcg = compute_ndcg(rec_dataset.validation_data, 
            rec_dataset.user_feature_matrix, 
            rec_dataset.item_feature_matrix, 
            rec_k, 
            model, 
            device)
            print('epoch %d: ' % epoch, 'training loss: ', ave_train, 'NDCG: ', ndcg)
    torch.save(model.state_dict(), os.path.join(out_path, "model_main.model"))
    ndcg_test = compute_ndcg(rec_dataset.test_data, 
            rec_dataset.user_feature_matrix, 
            rec_dataset.item_feature_matrix, 
            rec_k, 
            model, 
            device)
    print('NDCG Test: ',ndcg_test)
    return 0


if __name__ == "__main__":
    torch.manual_seed(0)
    np.random.seed(0)
    if gpu:
        os.environ["CUDA_VISIBLE_DEVICES"] =cuda
        print("Using CUDA",cuda)
    else:
        print("Using CPU")
    train_base_recommendation()

Using CUDA 0
init ndcg: 0.3028848819387577


  0%|          | 0/36 [00:00<?, ?it/s]

epoch 0:  training loss:  0.42214802


  3%|▎         | 1/36 [00:14<08:13, 14.09s/it]

epoch 0:  training loss:  0.42214802 NDCG:  0.49525104293724864
epoch 1:  training loss:  0.4099281


...

 97%|█████████▋| 35/36 [05:19<00:08,  8.90s/it]

epoch 34:  training loss:  0.2879885 NDCG:  0.5192458777452177
epoch 35:  training loss:  0.2832115


100%|██████████| 36/36 [05:28<00:00,  9.12s/it]

epoch 35:  training loss:  0.2832115 NDCG:  0.5170412818019532


NDCG Test:  0.5001901568632098


# Training ExpOptimization Model...

In [ ]:
dataset="yelp"
base_model_path='/content/drive/MyDrive/Yelp/'
gpu=True
cuda='0'
data_obj_path='/content/drive/MyDrive/Yelp/'
rec_k=5
lam=100
gam=1.0
alp=0.01
user_mask=False
lr=0.01
step=1000
mask_thresh=0.1
test_num=1000
save_path='/content/drive/MyDrive/Yelp/'

In [ ]:
import torch
import pickle
import os
from pathlib import Path

In [ ]:
save_path='/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/Yelp/'

In [ ]:
items_list1=[]
users_list=[]
review_features={}
f=open(save_path+'Yelp')
lines=f.readlines()
i=0
for line in lines:
  if i%100000==0:
    print(i)
  i+=1
  user_id = line.split('@')[0]
  item_id = line.split('@')[1]
  # if item_id in items_list:
  users_list.append(user_id)
  items_list1.append(item_id)
  l = len(user_id) + len(item_id)
  fosr_data = line[l+3:]
  for seg in fosr_data.split('||'):
    if (user_id,item_id) not in review_features.keys():
      review_features[(user_id,item_id)]=[]
    fos = seg.split(':')[0].strip('|')
    if len(fos.split('|')) > 1:
          feature = fos.split('|')[0]
          opinion = fos.split('|')[1]
          sentiment = fos.split('|')[2]
          sentence= seg.split(':')[1]
          if sentiment=='+1':
            senti=1
          else:
            senti=-1
          review_features[(user_id,item_id)].append([feature,opinion,senti,sentence])
    else:
      print(user_id,item_id)

0
100000
200000
300000
400000
500000
1Xw_npZXLcsWBvlLYCiW_A fKrmWy4GFsrgdOYhN9pyZA
600000
700000
800000
900000
1000000
1100000
1200000
1300000


In [ ]:
del review_features

In [ ]:
import json
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import FreqDist
import re
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def lemmatization(text):
    result=''
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
        # if pos in ['n','a']:   
        result+=wordnet.lemmatize(token,pos)+' '
    return result
def remove_stopwords(text):
    en_stopwords = stopwords.words('english')
    en_stopwords+=['may','could','that','without','iii','with','and','This','That','Those','These','the','The','brbr','so','it','such']
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)
            
    return result
def remove_punct(text):
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

def remove_tag(text):
    text=' '.join(text)
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def preprocess(text):
  chars=['&','%','#','@','^','>','<','\n','\\','\t',';','"','/']
  stwords=stopwords.words('english')
  for ch in chars:
    text=text.replace(ch,' ')
  text=" ".join(text.split())
  # text=text.lower()
  text_tokenized=word_tokenize(text)
  cleaned_text= remove_stopwords(text_tokenized)
  cleaned_text= remove_punct(cleaned_text)
  # cleaned_text=lemmatization(cleaned_text)
  cleaned_text=remove_tag(cleaned_text)
  cleaned_text=remove_urls(cleaned_text)
  cleaned_text=''.join([i for i in cleaned_text ])
  cleaned_text=[word for word in cleaned_text.split(' ') if len(word)>1]
  # print(cleaned_text)
  return ' '.join(cleaned_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def preprocess_text_first(text):
  while '<' in text and '>' in text and text.index('<')<text.index('>'):
    toRemove=text[int(text.index('<')):int(text.index('>'))]+'>'
    text=text.replace(toRemove,' ')
  list_to_replace=['mso','gte','xml','false','#',',','!','-','\'','\"','[',']','/','\\n','\\','span','a-size-base','a-color-secondary','input type','header name','value','=','<a href= javascript:void(0) class= ','{','}','class=','header','<a href= javascript:void(0)','<','>','href',')','(',';','quot','&',':','javascript']
  for char in list_to_replace:
    text=text.replace(char,' ')
  for i in range(15):
    text=text.replace('  ',' ')
  # while 'if' in text and 'endif' in text and text.index('if')<text.index('endif'):
  #   # print(int(text.index('if')),int(text.index('endif')))
  #   toRemove=text[int(text.index('if')):int(text.index('endif'))]+'endif'
  #   text=text.replace(toRemove,' ')
  new_text=''
  for word in text.split(' '):
    if len(word)>1 and len(word)<35:
      new_text+=word+' '
  # new_text=lemmatization(new_text)
  # print(new_text)
  return new_text

In [ ]:
user_test_perspective={}
i=0
for (user_id , item_id) in review_features.keys():
  review_feature=review_features[(user_id,item_id)]
  if i%50000==0:
    print(i)
  i+=1
  for features in review_feature:
    sentence=features[3]
    sentence=preprocess_text_first(sentence)
    sentence=preprocess(sentence).lower()
    final_vect=[]
    if (user_id , item_id) not in user_test_perspective.keys():
      new_final_vect=[]
    else:
      new_final_vect=user_test_perspective[(user_id , item_id)]
    final_vect+=sentence.split(' ')
    for word in sentence.split(' '):
      # tokens=list(set(df_words[df_words['word']==word]['tokenized'].values))
      # for token in tokens:
        final_vect+=word.split(' ')
    final_vect=list(set(final_vect))
    for word in final_vect:
      if len(word)>1:
        new_final_vect.append(word)
    user_test_perspective[(user_id , item_id)]=list(set(new_final_vect))

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000


In [ ]:
user_test_perspective

In [ ]:
def evaluate_user_perspective(user_perspective_data, u_i_expl_dict):
    pres = []
    recs = []
    f1s = []
    # print(user_perspective_data)
    # rint(u_i_expl_dict)
    for u_i, gt_features in user_perspective_data.items():
        # print(u_i)
        # print(u_i_expl_dict)
        if u_i in u_i_expl_dict:
            TP = 0
            pre_features = u_i_expl_dict[u_i]
            # print('f: ', gt_features, pre_features)
            for feature in pre_features:
                if feature in gt_features:
                    TP += 1
            pre = TP / len(pre_features)
            rec = TP / len(gt_features)
            if (pre + rec) != 0:
                f1 = (2 * pre * rec) / (pre + rec)
            else:
                f1 = 0
            pres.append(pre)
            recs.append(rec)
            f1s.append(f1)
    ave_pre = np.mean(pres)
    ave_rec = np.mean(recs)
    ave_f1 = np.mean(f1s)
    return ave_pre, ave_rec, ave_f1

In [ ]:
def evaluate_model_perspective(
        rec_dict,
        u_i_exp_dict,
        base_model,
        user_feature_matrix,
        item_feature_matrix,
        rec_k,
        device):
    """
    compute PN, PS and F_NS score for the explanations
    :param rec_dict: {u1: [i1, i2, i3, ...] , u2: [i1, i2, i3, ...]}
    :param u_i_exp_dict: {(u, i): [f1, f2, ...], ...}
    :param base_model: the trained base recommendation model
    :param user_feature_matrix: |u| x |p| matrix, the attention on each feature p for each user u
    :param item_feature_matrix: |i| x |p| matrix, the quality on each feature p for each item i
    :param rec_k: the length of the recommendation list, only generated explanations for the items on the list
    :param device: the device of the model
    :return: the mean of the PN, PS and FNS scores
    """
    pn_count = 0
    ps_count = 0
    for u_i, fs in u_i_exp_dict.items():
        user = u_i[0]
        target_item = u_i[1]
        features = set(fs)
        items = rec_dict[user]
        target_index = items.index(target_item)
        # compute PN
        cf_items_features = []
        for item in items:
            item_ori_feature = np.array(item_feature_matrix[item])
            item_cf_feature = np.array([0 if s in features else item_ori_feature[s]
                                        for s in range(len(item_ori_feature))], dtype='float32')
            cf_items_features.append(item_cf_feature)
        cf_ranking_scores = base_model(torch.from_numpy(np.array([user_feature_matrix[user]
                                                                      for i in range(len(cf_items_features))])
                                                            ).to(device),
                                           torch.from_numpy(np.array(cf_items_features)).to(device)).squeeze()
        cf_score_list = cf_ranking_scores.to('cpu').detach().numpy()
        sorted_index = np.argsort(cf_score_list)[::-1]
        cf_rank = np.argwhere(sorted_index == target_index)[0, 0]  # the updated ranking of the current item
        if cf_rank > rec_k - 1:
            pn_count += 1
        # compute NS
        cf_items_features = []
        for item in items:
            item_ori_feature = np.array(item_feature_matrix[item])
            item_cf_feature = np.array([item_ori_feature[s] if s in features else 0
                                        for s in range(len(item_ori_feature))], dtype='float32')
            cf_items_features.append(item_cf_feature)
        cf_ranking_scores = base_model(torch.from_numpy(np.array([user_feature_matrix[user]
                                                                      for i in range(len(cf_items_features))])
                                                            ).to(device),
                                           torch.from_numpy(np.array(cf_items_features)).to(device)).squeeze()
        cf_score_list = cf_ranking_scores.to('cpu').detach().numpy()
        sorted_index = np.argsort(cf_score_list)[::-1]
        cf_rank = np.argwhere(sorted_index == target_index)[0, 0]  # the updated ranking of the current item
        if cf_rank < rec_k:
            ps_count += 1
    if len(u_i_exp_dict) != 0:
        pn = pn_count / len(u_i_exp_dict)
        ps = ps_count / len(u_i_exp_dict)
        if (pn + ps) != 0:
            fns = (2 * pn * ps) / (pn + ps)
        else:
            fns = 0
    else:
        pn = 0
        ps = 0
        fns = 0
    return pn, ps, fns

In [ ]:
import json

In [ ]:
class ExpOptimizationModel(torch.nn.Module):
    def __init__(self, base_model, rec_dataset, device):
        super(ExpOptimizationModel, self).__init__()
        self.base_model = base_model
        self.rec_dataset = rec_dataset
        self.device = device
        self.u_i_exp_dict = {}  # {(user, item): [f1, f2, f3 ...], ...}
        self.user_feature_matrix = torch.from_numpy(self.rec_dataset.user_feature_matrix).to(self.device)
        self.item_feature_matrix = torch.from_numpy(self.rec_dataset.item_feature_matrix).to(self.device)
        self.rec_dict, self.user_perspective_test_data = self.generate_rec_dict()

    def generate_rec_dict(self):
        rec_dict = {}
        correct_rec_dict = {}  # used for user-side evaluation
        for row in self.rec_dataset.test_data:
            user = row[0]
            items = row[1]
            labels = row[2]
            correct_rec_dict[user] = []
            user_features = self.user_feature_matrix[user].repeat(len(items), 1)
            scores = self.base_model(user_features,
                        self.item_feature_matrix[items]).squeeze()
            scores = np.array(scores.to('cpu'))
            sort_index = sorted(range(len(scores)), key=lambda k: scores[k], reverse=True)
            sorted_items = [items[i] for i in sort_index]
            rec_dict[user] = sorted_items
            for i in range(rec_k):  # find the correct items and add to the user side test data
                if labels[sort_index[i]] == 1:
                    correct_rec_dict[user].append(items[sort_index[i]])
        user_item_feature_dict = {}  # {(u, i): f, (u, i): f]

        for row in self.rec_dataset.sentiment_data:
            user = row[0]
            item = row[1]
            user_item_feature_dict[(user, item)] = []
            for fos in row[2:]:
                feature = fos[0]
                user_item_feature_dict[(user, item)].append(feature)
        user_perspective_test_data = {}  # {(u, i):f, (u, i): f]}
        # for user, tiems in correct_rec_dict.items():
        #     for item in tiems:
        #       if (user, item) in user_item_feature_dict.keys():
        #           feature = user_item_feature_dict[(user, item)]
        #           user_perspective_test_data[(user, item)] = feature
        # # print(user_perspective_test_data)
        return rec_dict, user_perspective_test_data

    def generate_explanation(self):
        # u_i_exps_dict = {}  # {(user, item): [f1, f2, f3 ...], ...}
        exp_nums = []
        exp_complexities = []
        self.no_exp_count = 0
        self.all_count=0
        if test_num == -1:
            test_num1 = len(list(self.rec_dict.items()))
        else:
            test_num1 = test_num
        # print(test_num1)
        count=0
        for user, items in tqdm.tqdm(list(self.rec_dict.items())[:10]):
            count+=1
            items = self.rec_dict[user]
            margin_item = items[rec_k]
            margin_score = self.base_model(self.user_feature_matrix[user].unsqueeze(0), 
                            self.item_feature_matrix[margin_item].unsqueeze(0)).squeeze()
            if user_mask:
                # mask_vec = self.generate_mask(user)
                mask_vec = torch.where(self.user_feature_matrix[user]>0, 1., 0.).unsqueeze(0)  # only choose exps from the user cared aspects
            else:
                mask_vec = torch.ones(self.rec_dataset.feature_num, device=self.device).unsqueeze(0)
            for item in items[: rec_k]:
                explanation_features, exp_num, exp_complexity = self.explain(
                    self.user_feature_matrix[user], 
                    self.item_feature_matrix[item], 
                    margin_score,
                    mask_vec)
                self.all_count+=1
                if explanation_features is None:
                    # print('no explanation for user %d and item %d' % (user, item))
                    self.no_exp_count += 1
                else:
                    self.u_i_exp_dict[(user, item)] = explanation_features
                    exp_nums.append(exp_num)
                    exp_complexities.append(exp_complexity)
            # if count%500==0:
            #   json1 = json.dumps(self.u_i_exp_dict)
            #   f = open("drive/MyDrive/ranjbar/dict{}.json".format(count),"w")
            #   f.write(json1)
            #   f.close()

        print('all_count',self.all_count,'ave num: ', np.mean(exp_nums), 'ave complexity: ', np.mean(exp_complexities) , 'no_exp_count: ', self.no_exp_count)
        # print()
        return True
    
    def explain(self, user_feature, item_feature, margin_score, mask_vec):
        exp_generator = EXPGenerator(
            self.rec_dataset, 
            self.base_model, 
            user_feature, 
            item_feature, 
            margin_score, 
            mask_vec,
            self.device).to(self.device)

        # optimization
        optimizer = torch.optim.SGD(exp_generator.parameters(), lr=lr, weight_decay=0)
        exp_generator.train()
        lowest_loss = None
        lowest_bpr = None
        lowest_l2 = 0
        optimize_delta = None
        score = exp_generator()
        bpr, l2, l1, loss = exp_generator.loss(score)
        # print('init: ', 0, '  train loss: ', loss, '  bpr: ', bpr, '  l2: ', l2, '  l1: ', l1)
        lowest_loss = loss
        optimize_delta = exp_generator.delta.detach().to('cpu').numpy()
        lowest_l2 = l2
        for epoch in range(step):
            exp_generator.zero_grad()
            score = exp_generator()
            bpr, l2, l1, loss = exp_generator.loss(score)
            # if epoch % 100 == 0:
            #     print(
            #         'epoch', epoch,
            #         'bpr: ', bpr,
            #         'l2: ', l2,
            #         'l1', l1,
            #         'loss', loss)

            loss.backward()
            optimizer.step()
            if loss < lowest_loss:
                lowest_loss = loss
                lowest_l2 = l2
                lowest_bpr = bpr
                optimize_delta = exp_generator.delta.detach().to('cpu').numpy()
        if lowest_bpr >= lam * alp:
            explanation_features = None 
            exp_num = None
            exp_complexity = None
        else:
            # optimize_delta = exp_generator.delta.detach().to('cpu').numpy()
            explanation_features = np.argwhere(optimize_delta < - mask_thresh).squeeze(axis=1)
            if len(explanation_features) == 0:
                explanation_features = np.array([np.argmin(optimize_delta)])
            exp_num = len(explanation_features)
            exp_complexity = lowest_l2.to('cpu').detach().numpy() + gam * exp_num
        return explanation_features, exp_num, exp_complexity
    
    def user_side_evaluation(self):
        ave_pre, ave_rec, ave_f1 = evaluate_user_perspective(self.user_perspective_test_data, self.u_i_exp_dict)
        print('user\'s perspective:')
        print('ave pre: ', ave_pre, '  ave rec: ', ave_rec, '  ave f1: ', ave_f1)
    
    def model_side_evaluation(self):
        ave_pn, ave_ps, ave_fns = evaluate_model_perspective(
            self.rec_dict,
            self.u_i_exp_dict,
            self.base_model,
            self.rec_dataset.user_feature_matrix,
            self.rec_dataset.item_feature_matrix,
            rec_k,
            self.device)
        print('model\'s perspective:')
        print('ave PN: ', ave_pn, '  ave PS: ', ave_ps, '  ave F_{NS}: ', ave_fns)  


In [ ]:
class EXPGenerator(torch.nn.Module):
    def __init__(self, rec_dataset, base_model, user_feature, item_feature, margin_score, mask_vec, device):
        super(EXPGenerator, self).__init__()
        self.rec_dataset = rec_dataset
        self.base_model = base_model
        self.user_feature = user_feature
        self.item_feature = item_feature
        self.margin_score = margin_score
        self.mask_vec = mask_vec
        self.device = device
        self.feature_range = [0, 5]  # hard coded, should be improved later
        self.delta_range = self.feature_range[1] - self.feature_range[0]  # the maximum feature value.
        self.delta = torch.nn.Parameter(
            torch.FloatTensor(len(self.user_feature)).uniform_(-self.delta_range, 0))

    def get_masked_item_feature(self):
        item_feature_star = torch.clamp(
            (self.item_feature + torch.clamp((self.delta * self.mask_vec), -self.delta_range, 0)),
            self.feature_range[0], self.feature_range[1])
        # print(self.item_feature)
        # print(self.delta)
        return item_feature_star
    
    def forward(self):
        item_feature_star = self.get_masked_item_feature()
        # print(item_feature_star)
        score = self.base_model(self.user_feature.unsqueeze(0), item_feature_star)
        return score
    
    def loss(self, score):
        bpr = torch.nn.functional.relu(alp + score - self.margin_score) * lam
        # print(score)
        # print(self.margin_score)
        l2 = torch.linalg.norm(self.delta)
        l1 = torch.linalg.norm(self.delta, ord=1) * gam
        loss = l2 + bpr + l1
        return bpr, l2, l1, loss

In [ ]:
def generate_explanation():
    if gpu:
        device = torch.device('cuda:%s' %cuda)
    else:
        device = 'cpu'
    print(device)
    # import dataset
    # with open(os.path.join(data_obj_path, dataset + "_dataset_obj_main.pickle"), 'rb') as inp:
    #     rec_dataset = pickle.load(inp)
    
    base_model = BaseRecModel(rec_dataset.feature_num).to(device)
    base_model.load_state_dict(torch.load(os.path.join(base_model_path,"model_main.model"),map_location=torch.device(device)))
    base_model.eval()
    #  fix the rec model
    for param in base_model.parameters():
        param.requires_grad = False
    
    # Create optimization model
    opt_model = ExpOptimizationModel(
        base_model=base_model,
        rec_dataset=rec_dataset,
        device = device,
        
    )

    opt_model.generate_explanation()
    opt_model.user_side_evaluation()
    opt_model.model_side_evaluation()
    # print(opt_model.u_i_exp_dict)
    # Path(save_path).mkdir(parents=True, exist_ok=True)
    # with open(os.path.join(save_path, dataset + "_explanation_obj_main.pickle"), 'wb') as outp:
    #     pickle.dump(opt_model, outp, pickle.HIGHEST_PROTOCOL)
    return opt_model


if __name__ == "__main__":
    opt_model=generate_explanation()

cuda:0


100%|██████████| 200/200 [22:44<00:00,  6.82s/it]
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


all_count 1000 ave num:  6.567421790722761 ave complexity:  10.501755286880167 no_exp_count:  73
user's perspective:
ave pre:  nan   ave rec:  nan   ave f1:  nan
model's perspective:
ave PN:  0.9719525350593312   ave PS:  0.9320388349514563   ave F_{NS}:  0.9515773261090789


cuda:0

100%|██████████| 1000/1000 [56:18<00:00,  3.38s/it]

all_count 5000
user's perspective:
ave pre:  0.055013481725532325   ave rec:  0.24509021707792805   ave f1:  0.07864064144286358
model's perspective:
ave PN:  0.9956788719581533   ave PS:  0.9563338639981805   ave F_{NS}:  0.9756098465767294


In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
with open(os.path.join(save_path2, dataset + "_dataset_obj_onlytips.pickle"), 'rb') as inp:
  rec_dataset2 = pickle.load(inp)

In [ ]:
rec_dataset2.test_data

In [ ]:
chosen_list=[(90027, 1379),(50, 4398),(7063, 13200),(1120, 1321),(1097, 13448),(6708, 12469),(6708, 2261),(8694, 6988),(1717, 3766)]

In [ ]:
user_item_ids=[]
for (user,item) in chosen_list:
  user_id=rec_dataset2.inv_user_name_dict[user]
  item_id=rec_dataset2.inv_item_name_dict[item]
  user_item_ids.append((user_id,item_id))

In [ ]:
inv_user_name_dict = {v: k for k, v in rec_dataset.user_name_dict.items()}
inv_item_name_dict = {v: k for k, v in rec_dataset.item_name_dict.items()}
inv_feature_name_dict = {v: k for k, v in rec_dataset.feature_name_dict.items()}

In [ ]:
for (user,item) in opt_model.u_i_exp_dict.keys():
  user_id=inv_user_name_dict[user]
  item_id=inv_item_name_dict [item]
  if(user_id,item_id) in user_item_ids:
    print(user_id,item_id)
    feas= opt_model.u_i_exp_dict[(user,item)]
    for fea in feas:
      print(inv_feature_name_dict[fea])
    print('----------')

pCpZ6KF0vCOF3fyj9W8iMg 7JhQXpiMml41sxN34CUcKQ
store
beans
sugar
neighborhood
bathroom
stores
----------
nlReKgQoRz6uPfVaEG93mw tU692E8N0xBQ7Ogc78gN2g
staff
taste
inside
----------
0du93EkEwKuxRG_x6hqVUg KnsY8rh5tigp5t6WpilGdA
coffee
favorite
spot
cheese
tasting
price
eating
hour
ingredients
tea
chocolate
neighborhood
shop
----------


In [ ]:
sentiment_data = []  # [userID, itemID, [fos triplet 1], [fos triplet 2], ...]
with open(sentires_dir, 'r') as f:
    line = f.readline().strip()
    while line:
        # print(count)
        # print('line', line)
        user = line.split('@')[0]
        item = line.split('@')[1]
        sentiment_data.append([user, item])
        l = len(user) + len(item)
        fosr_data = line[l+3:]
        for seg in fosr_data.split('||'):
            fos = seg.split(':')[0].strip('|')
            if len(fos.split('|')) > 1:
                feature = fos.split('|')[0]
                opinion = fos.split('|')[1]
                sentiment = fos.split('|')[2]
                sentiment_data[-1].append([feature, opinion, sentiment])
        line = f.readline().strip()
sentiment_data = np.array(sentiment_data)
sentiment_data = sentiment_data_filtering(
    sentiment_data, 
    user_thresh, 
    feature_thresh)
user_dict, item_dict = get_user_item_dict(sentiment_data)  # not sorted with time
user_item_date_dict = {}   # {(user, item): date, (user, item): date ...}  # used to remove duplicate

for i, line in enumerate(open(review_dir, "r")):
    record = json.loads(line)
    user = record['reviewerID']
    item = record['asin']
    date = record['unixReviewTime']
    if user in user_dict and item in user_dict[user] and (user, item) not in user_item_date_dict:
        user_item_date_dict[(user, item)] = date

# remove the (user, item) not exist in the official dataset, possibly due to update?
sentiment_data = [row for row in sentiment_data if (row[0], row[1]) in user_item_date_dict]
sentiment_data = sentiment_data_filtering(sentiment_data, user_thresh, feature_thresh)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


======================= filtering sentiment data =======================
original review length:  52178
original user length:  21615
original item length:  9292
original feature length:  325


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


valid review length:  4454
valid user:  251
valid item :  1918
valid feature length:  88
user dense is: 17.745019920318725
======================= filtering sentiment data =======================
original review length:  1072
original user length:  201
original item length:  634
original feature length:  84
valid review length:  180
valid user:  14
valid item :  134
valid feature length:  15
user dense is: 12.857142857142858


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
user_name_dict = {}
item_name_dict = {}
feature_name_dict = {}
features = get_feature_list(sentiment_data)
count = 0
for feature in features:
    if feature not in feature_name_dict:
        feature_name_dict[feature] = count
        count += 1
count = 0
for user in user_dict:
    if user not in user_name_dict:
        user_name_dict[user] = count
        count += 1
count = 0
for item in item_dict:
    if item not in item_name_dict:
        item_name_dict[item] = count
        count += 1

In [ ]:
len(user_dict)

251

In [ ]:
inv_user_name_dict = {v: k for k, v in user_name_dict.items()}
inv_item_name_dict = {v: k for k, v in item_name_dict.items()}
inv_features_dict = {v: k for k, v in feature_name_dict.items()}

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Cell_Phones_and_Accessories.json.gz

--2022-03-06 15:23:24--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Cell_Phones_and_Accessories.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1232323281 (1.1G) [application/octet-stream]
Saving to: ‘Cell_Phones_and_Accessories.json.gz’

Cell_Phones_and_Acc 100%[===================>]   1.15G  21.4MB/s    in 56s     

2022-03-06 15:24:21 (20.9 MB/s) - ‘Cell_Phones_and_Accessories.json.gz’ saved [1232323281/1232323281]



In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    name=b'"verified": \"true\",'
    l=l.replace(b'"verified": true,',bytes(name))
    name1=b'"verified": \"false\",'
    l=l.replace(b'"verified": false,',bytes(name))
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    rem_list={'verified','reviewTime','style','image'}
    [d.pop(key) for key in rem_list if key in d.keys()]
    if d['reviewerID'] in user_dict:
      df[i] = d
      i += 1
      if i%1000==0:
        print(i)
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
df_reviews = getDF('Cell_Phones_and_Accessories.json.gz')

1000
2000
3000
4000
5000
6000
7000
8000
9000


In [ ]:
df_reviews['asin']

In [ ]:
for key in opt_model.u_i_exp_dict.keys():
  user=key[0]
  item=key[1]
  features=opt_model.u_i_exp_dict[key]
  user_id=inv_user_name_dict[user]
  item_id=inv_item_name_dict[item]
  print(user_id)
  print(item_id)
  # print(df_reviews[(df_reviews['reviewerID']==user_id)])
  for feature in features:
    print(inv_features_dict[feature])
  print('----------------')

AL4YOB7KRVQ9W
B0013URK04
battery
size
----------------
AL4YOB7KRVQ9W
B00170KUM0
case
----------------
AL4YOB7KRVQ9W
B000FL9QGI
screen
size
----------------
AL4YOB7KRVQ9W
B000C1CHVC
screen
----------------
AL4YOB7KRVQ9W
B000SZ9I0K
case
----------------
A5JLAU2ARJ0BO
B00170KUM0
phone
----------------
A5JLAU2ARJ0BO
B000SZ9I0K
phone
----------------
A5JLAU2ARJ0BO
B000GAO9T2
phone
----------------
A5JLAU2ARJ0BO
B000C1CHVC
phone
screen
----------------
A22S7D0LP8GRDH
B0009W8DKI
phone
product
----------------
A22S7D0LP8GRDH
B000T4LFCO
size
----------------
A22S7D0LP8GRDH
B000SZ9I0K
phone
fit
----------------
A22S7D0LP8GRDH
B0000WZWSI
fit
----------------
A22S7D0LP8GRDH
B001BZH2QI
sound
battery
----------------
A1ODOGXEYECQQ8
B0013URK04
battery
----------------
A1ODOGXEYECQQ8
B000HJC56G
phone
----------------
A1ODOGXEYECQQ8
B000WR81ZM
battery
----------------
A1ODOGXEYECQQ8
B000XQGJUG
phone
----------------
A1ODOGXEYECQQ8
B0009W8DL2
battery
----------------
A1X1CEGHTHMBL1
B000BYPLVI
product
--

In [ ]:
def generate_explanation_check_stability():
    if gpu:
        device = torch.device('cuda:%s' %cuda)
    else:
        device = 'cpu'
    print(device)
    # import dataset
    # with open(os.path.join(data_obj_path, dataset + "_dataset_obj_main.pickle"), 'rb') as inp:
    #     rec_dataset = pickle.load(inp)
    
    base_model = BaseRecModel(rec_dataset.feature_num).to(device)
    base_model.load_state_dict(torch.load(os.path.join(base_model_path,"model_main.model"),map_location=torch.device(device)))
    base_model.eval()
    #  fix the rec model
    for param in base_model.parameters():
        param.requires_grad = False
    
    
    # Create optimization model
    features_found=[]
    for i in range(10):
      opt_model = ExpOptimizationModel(
        base_model=base_model,
        rec_dataset=rec_dataset,
        device = device,)
      opt_model.generate_explanation()
      features_found.append(opt_model.u_i_exp_dict)
      
    
    return features_found


if __name__ == "__main__":
    features_found=generate_explanation_check_stability()

cuda:0
cuda:0


100%|██████████| 10/10 [01:13<00:00,  7.32s/it]


all_count 50 ave num:  6.16 ave complexity:  10.002760995328426 no_exp_count:  0


100%|██████████| 10/10 [01:07<00:00,  6.72s/it]


all_count 50 ave num:  5.92 ave complexity:  9.768209390342236 no_exp_count:  0


...

all_count 50 ave num:  6.1 ave complexity:  9.98515693962574 no_exp_count:  0


100%|██████████| 10/10 [01:03<00:00,  6.39s/it]


all_count 50 ave num:  6.04 ave complexity:  9.931369032263756 no_exp_count:  0


100%|██████████| 10/10 [01:04<00:00,  6.43s/it]

all_count 50 ave num:  5.938775510204081 ave complexity:  9.787047523929148 no_exp_count:  1


In [ ]:
dict_features={}
for iter_feas in features_found:
  for u_i in iter_feas.keys():
    # feas=list(iter_feas[u_i][0][0].columns)
    feas=iter_feas[u_i]
    # print(iter_feas[u_i])
    if u_i in dict_features.keys():
      dict_features[u_i].append(feas)
    else:
      dict_features[u_i]=[]
      dict_features[u_i].append(feas)

In [ ]:
dict_features

# Calculate Stability...

In [ ]:
stability=0
# count_all=0
for ui in dict_features.keys():
  features=dict_features[ui]
  stabs=0
  count=0
  if(len(features)>1):
    # count_all+=1
    for i in range(len(features)):
      for j in range(len(features)):
        # print(features[i])
        # print(features[j])
        if len(features[i])>0 and len(features[j])>0:
          if i != j:
            intersection = list(set(features[i]) & set(features[j]))
            union = list(set(features[i]) | set(features[j]))
            # print(features[i],features[j])
            # print(intersection)
            # print(union)
            count+=1
            stabs+=(len(intersection)/len(union))
    # print(stabs)
    # print(len(features)*(len(features)-1))
    # print((stabs/(len(features)*(len(features)-1))))
    stability+=(stabs/(9.0*10.0))

stability=stability/len(dict_features)
print(stability)

0.7434970227211033
